# Flight Search Examples with Google Flights API

This notebook demonstrates how to use the Google Flights Search Tool with various search parameters and scenarios using SerpApi.

## Overview

The Flight Search Tool allows you to:
- Search for round-trip and one-way flights
- Specify passenger counts (adults, children, infants)
- Filter by travel class, price, time preferences
- Control layover durations
- Include/exclude specific airlines
- Enable deep search for more accurate results

Let's explore different examples of how to use this tool!

## 1. Import Required Libraries

First, let's import all the necessary libraries including `dotenv` for loading environment variables.

## 2. Load Environment Variables with dotenv

We'll use `load_dotenv()` to load the API key from our `.env` file. This is a secure way to manage sensitive credentials.

In [42]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from typing import Optional, Dict, List

# Verify that the API key is available
api_key = os.getenv('SERPAPI_API_KEY')
if api_key:
    print("✅ SERPAPI_API_KEY loaded successfully!")
    print(f"API Key preview: {api_key[:10]}...{api_key[-4:]}")
else:
    print("❌ SERPAPI_API_KEY not found. Please check your .env file.")
    print("Make sure you have a .env file with: SERPAPI_API_KEY=your_api_key_here")

✅ SERPAPI_API_KEY loaded successfully!
API Key preview: d38b412680...d3d9


## Important Note: Directory Structure

Since this notebook is in the `notebooks/` subdirectory and the `flight_search.py` file is in the parent directory (`travel-agent/`), we need to handle the import path correctly. 

There are several ways to solve this:

1. **Path manipulation** (used above): Add the parent directory to Python's path
2. **Copy the module**: Create a copy of `flight_search.py` in the notebooks directory  
3. **Relative imports**: Use relative import syntax
4. **Package structure**: Convert the project to a proper Python package

For this notebook, we're using approach #1 and also including the complete implementation to ensure everything works smoothly.

## 3. Define the FlightSearcher Class

Now let's define our FlightSearcher class that handles all the flight search functionality using the Google Flights API via SerpApi.

In [47]:
import requests


class FlightSearcher:
    """A class to search for flights using the SerpApi Google Flights API."""
    
    def __init__(self, api_key: str = None):
        """
        Initialize the FlightSearcher.
        
        Args:
            api_key (str): SerpApi API key. If not provided, will try to load from environment.
        """
        self.api_key = api_key or self._load_api_key()
        #self.base_url = "https://serpapi.com/search.json"
        self.base_url = "https://serpapi.com/search"
        
    def _load_api_key(self) -> str:
        """Load API key from environment or .env file."""
        # Load environment variables from .env file
        load_dotenv(override=True)

        # Try environment variable first
        api_key = os.getenv('SERPAPI_API_KEY')
        if api_key:
            return api_key
                
        raise ValueError("API key not found. Please set SERPAPI_API_KEY in .env file or environment variable.")
    
    def search_flights(self,
                      departure_id: str,
                      arrival_id: str,
                      outbound_date: str,
                      return_date: str = None,
                      trip_type: str = "round_trip",
                      adults: int = 1,
                      children: int = 0,
                      infants: int = 0,
                      travel_class: str = "economy",
                      departure_time_range: str = None,
                      return_time_range: str = None,
                      max_layover_duration: int = None,
                      min_layover_duration: int = None,
                      max_price: int = None,
                      include_airlines: list = None,
                      exclude_airlines: list = None,
                      max_duration: int = None,
                      stops: int = None,  # NEW: number of stops
                      currency: str = "USD",
                      language: str = "en",
                      country: str = "us",
                      deep_search: bool = False,
                      auto_fetch_return_flights: bool = True) -> dict:
        """
        Search for flights with specified parameters.
        
        Args:
            departure_id (str): Departure airport code (e.g., 'LAX', 'JFK')
            arrival_id (str): Arrival airport code (e.g., 'LHR', 'CDG')
            outbound_date (str): Departure date in YYYY-MM-DD format
            return_date (str, optional): Return date in YYYY-MM-DD format (required for round trip)
            trip_type (str): 'round_trip', 'one_way', or 'multi_city'
            adults (int): Number of adult passengers (default: 1)
            children (int): Number of child passengers (default: 0)
            infants (int): Number of infant passengers (default: 0)
            travel_class (str): 'economy', 'premium_economy', 'business', or 'first'
            departure_time_range (str): Time range for departure (e.g., '6,12' for 6AM-12PM)
            return_time_range (str): Time range for return flight
            max_layover_duration (int): Maximum layover duration in minutes
            min_layover_duration (int): Minimum layover duration in minutes
            max_price (int): Maximum ticket price
            include_airlines (List[str]): List of airline codes to include
            exclude_airlines (List[str]): List of airline codes to exclude
            max_duration (int): Maximum flight duration in minutes
            stops (int): Number of stops (0=nonstop, 1=one stop, etc.)
            currency (str): Currency code (default: 'USD')
            language (str): Language code (default: 'en')
            country (str): Country code (default: 'us')
            deep_search (bool): Enable deep search for more accurate results
            auto_fetch_return_flights (bool): Automatically fetch return flights for round-trip searches
        
        Returns:
            Dict: Flight search results from SerpApi with return flights included for round-trip searches
        """
        
        # Get outbound flights using the single search method
        outbound_results = self._search_flights_single(
            departure_id=departure_id,
            arrival_id=arrival_id,
            outbound_date=outbound_date,
            return_date=return_date,
            trip_type=trip_type,
            adults=adults,
            children=children,
            infants=infants,
            travel_class=travel_class,
            departure_time_range=departure_time_range,
            return_time_range=return_time_range,
            max_layover_duration=max_layover_duration,
            min_layover_duration=min_layover_duration,
            max_price=max_price,
            include_airlines=include_airlines,
            exclude_airlines=exclude_airlines,
            max_duration=max_duration,
            stops=stops,  # Pass stops to API call
            currency=currency,
            language=language,
            country=country,
            deep_search=deep_search
        )
        
        # For round-trip flights, automatically fetch return flights if enabled
        if (trip_type == "round_trip" and auto_fetch_return_flights and 
            not outbound_results.get("error")):
            enhanced_results = self._fetch_complete_round_trip(outbound_results)
            return enhanced_results
        
        return outbound_results
    
    def _search_flights_single(self, **kwargs) -> dict:
        """Internal method to perform a single flight search API call."""
        # Build the parameters dictionary
        params = {
            "engine": "google_flights",
            "departure_id": kwargs["departure_id"].upper(),
            "arrival_id": kwargs["arrival_id"].upper(),
            "outbound_date": kwargs["outbound_date"],
            "currency": kwargs.get("currency", "USD"),
            "hl": kwargs.get("language", "en"),
            "gl": kwargs.get("country", "us"), #TODO how about international search?
            "api_key": self.api_key
        }
        
        # Set trip type
        type_mapping = {
            "round_trip": 1,
            "one_way": 2,
            "multi_city": 3
        }
        params["type"] = type_mapping.get(kwargs.get("trip_type", "round_trip"), 1)
        
        # Add return date for round trip
        if kwargs.get("trip_type") == "round_trip":
            if not kwargs.get("return_date"):
                raise ValueError("Return date is required for round trip flights")
            params["return_date"] = kwargs["return_date"]
        elif kwargs.get("return_date") and kwargs.get("trip_type") == "one_way":
            print("Warning: Return date specified for one-way trip. Ignoring return date.")
        
        # Set travel class
        class_mapping = {
            "economy": 1,
            "premium_economy": 2,
            "business": 3,
            "first": 4
        }
        params["travel_class"] = class_mapping.get(str(kwargs.get("travel_class", "economy")), 1)

        # Set passenger counts
        if kwargs.get("adults", 1) > 0:
            params["adults"] = str(kwargs["adults"])
        if kwargs.get("children", 0) > 0:
            params["children"] = str(kwargs["children"])
        if kwargs.get("infants", 0) > 0:
            params["infants_in_seat"] = str(kwargs["infants"])

        # Set time preferences
        if kwargs.get("departure_time_range"):
            params["outbound_times"] = kwargs["departure_time_range"]
        if kwargs.get("return_time_range") and kwargs.get("trip_type") == "round_trip":
            params["return_times"] = kwargs["return_time_range"]
            
        # Set layover preferences
        if kwargs.get("min_layover_duration") and kwargs.get("max_layover_duration"):
            params["layover_duration"] = f"{kwargs['min_layover_duration']},{kwargs['max_layover_duration']}"
        
        # Set price and duration limits
        if kwargs.get("max_price"):
            params["max_price"] = kwargs["max_price"]
        if kwargs.get("max_duration"):
            params["max_duration"] = kwargs["max_duration"]
            
        # Set airline preferences
        if kwargs.get("include_airlines"):
            params["include_airlines"] = ",".join(kwargs["include_airlines"])
        if kwargs.get("exclude_airlines"):
            params["exclude_airlines"] = ",".join(kwargs["exclude_airlines"])
        # Set stops filter
        #NOTE added on 9-5
        if kwargs.get("stops") is not None:
            stops = kwargs["stops"]
            if isinstance(stops, str) and stops.lower() == "nonstop":
                stops = 1
            if stops not in (0, 1, 2, 3):
                raise ValueError("stops must be one of 0 (any), 1 (nonstop), 2 (≤1 stop), 3 (≤2 stops).")
            params["stops"] = stops
        
        # Enable deep search if requested
        if kwargs.get("deep_search"):
            params["deep_search"] = "true"
        
        # Make the API request
        #response = requests.get(self.base_url, params=params)
        #print(f"[DEBUG] URL: {response.url}")
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            body = e.response.text if getattr(e, "response", None) is not None else ""
            raise Exception(f"HTTP {e.response.status_code} for {response.url}\n{body}") from e
        
    
    def _fetch_complete_round_trip(self, outbound_results: Dict) -> Dict:
        """
        Fetch return flights for round-trip searches and combine with outbound flights.
        For each outbound flight, combine with ALL available return flight options.
        """
        all_flights = []
        if outbound_results.get("best_flights"):
            all_flights.extend(outbound_results["best_flights"])
        if outbound_results.get("other_flights"):
            all_flights.extend(outbound_results["other_flights"])
        enhanced_flights = []
        for flight in all_flights:
            if flight.get("departure_token"):
                try:
                    return_results = self._fetch_return_flights(
                        flight["departure_token"],
                        outbound_params=outbound_results.get("search_parameters", {})
                    )
                    combos = self._combine_outbound_all_return_flights(flight, return_results)
                    enhanced_flights.extend(combos)
                except Exception:
                    enhanced_flights.append(flight)
            else:
                enhanced_flights.append(flight)
        enhanced_results = outbound_results.copy()
        num_best = len(outbound_results.get("best_flights", []))
        if num_best > 0:
            enhanced_results["best_flights"] = enhanced_flights[:num_best]
            if len(enhanced_flights) > num_best:
                enhanced_results["other_flights"] = enhanced_flights[num_best:]
        else:
            enhanced_results["other_flights"] = enhanced_flights
        return enhanced_results

    def _fetch_return_flights(self, departure_token: str, outbound_params: dict = None) -> Dict:
        """
        Fetch return flights using a departure token, using multi_city_json and type=3 for round-trip.
        
        Args:
            departure_token (str): Token from outbound flight for fetching return flights
            outbound_params (dict): Original search params for outbound leg (optional, for multi-city_json)
            
        Returns:
            Dict: Return flight search results
        """
        # If outbound_params provided, build multi_city_json for round-trip
        params = {
            "engine": "google_flights",
            "api_key": self.api_key,
            "departure_token": departure_token
        }
        if outbound_params and outbound_params.get("return_date") and outbound_params.get("departure_id") and outbound_params.get("arrival_id"):
            params["type"] = 3
            params["currency"] = outbound_params.get("currency", "USD")
            params["hl"] = outbound_params.get("language", "en")
            # params["gl"] = outbound_params.get("country", "us")
            # Build multi_city_json for round-trip
            multi_city_json = [
                {"departure_id": outbound_params["departure_id"], "arrival_id": outbound_params["arrival_id"], "date": outbound_params["outbound_date"]},
                {"departure_id": outbound_params["arrival_id"], "arrival_id": outbound_params["departure_id"], "date": outbound_params["return_date"]}
            ]
            import json as _json
            params["multi_city_json"] = _json.dumps(multi_city_json)
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"[ERROR] HTTPError fetching return flights: {e}")
            if e.response is not None:
                print(f"[ERROR] Response content: {e.response.text}")
            raise Exception(f"Return flight API request failed: {e}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] RequestException fetching return flights: {e}")
            raise Exception(f"Return flight API request failed: {e}")
    
    def _combine_outbound_return_flights(self, outbound_flight: Dict, return_results: Dict) -> Dict:
        """
        Combine outbound flight with return flight options to create complete round-trip data.
        
        Args:
            outbound_flight (Dict): Outbound flight data
            return_results (Dict): Return flight search results
            
        Returns:
            Dict: Combined flight data with both outbound and return segments
        """
        
        # Start with the outbound flight as base
        combined_flight = outbound_flight.copy()
        
        # Find the best return flight option (first in best_flights or other_flights)
        return_flight = None
        if return_results.get("best_flights"):
            return_flight = return_results["best_flights"][0]
        elif return_results.get("other_flights"):
            return_flight = return_results["other_flights"][0]
        else:
            print("🔧 WARNING: No return flights found in results")
        
        if return_flight and return_flight.get("flights"):
            # Combine the flight segments
            outbound_segments = combined_flight.get("flights", [])
            return_segments = return_flight.get("flights", [])
            
            # Update the combined flight
            combined_flight["flights"] = outbound_segments + return_segments
            
            # Update total duration if both are available
            outbound_duration = combined_flight.get("total_duration", 0)
            return_duration = return_flight.get("total_duration", 0)
            if outbound_duration and return_duration:
                combined_flight["total_duration"] = outbound_duration + return_duration
            
            # Combine layovers
            outbound_layovers = combined_flight.get("layovers", [])
            return_layovers = return_flight.get("layovers", [])
            combined_flight["layovers"] = outbound_layovers + return_layovers
            
            # Add return flight info for debugging
            combined_flight["_return_flight_info"] = {
                "return_price": return_flight.get("price"),
                "return_currency": return_flight.get("currency"),
                "return_duration": return_flight.get("total_duration"),
                "return_segments_count": len(return_segments)
            }
            
        else:
            print("🔧 WARNING: No return flight segments to combine")
        
        return combined_flight
    
    def _combine_outbound_all_return_flights(self, outbound_flight: Dict, return_results: Dict) -> list:
        """
        Combine outbound flight with ALL return flight options to create all possible round-trip combinations.
        Returns a list of combined flights, each with the correct price/currency for that combination.
        """
        combos = []
        return_flights = []
        if return_results.get("best_flights"):
            return_flights.extend(return_results["best_flights"])
        if return_results.get("other_flights"):
            return_flights.extend(return_results["other_flights"])
        if not return_flights:
            print("🔧 WARNING: No return flights found in results (all return options)")
            return [outbound_flight]
        for idx, return_flight in enumerate(return_flights):
            if return_flight.get("flights"):
                combined_flight = outbound_flight.copy()
                outbound_segments = combined_flight.get("flights", [])
                return_segments = return_flight.get("flights", [])
                combined_flight["flights"] = outbound_segments + return_segments
                outbound_duration = combined_flight.get("total_duration", 0)
                return_duration = return_flight.get("total_duration", 0)
                if outbound_duration and return_duration:
                    combined_flight["total_duration"] = outbound_duration + return_duration
                outbound_layovers = combined_flight.get("layovers", [])
                return_layovers = return_flight.get("layovers", [])
                combined_flight["layovers"] = outbound_layovers + return_layovers
                combined_flight["_return_flight_info"] = {
                    "return_price": return_flight.get("price"),
                    "return_currency": return_flight.get("currency"),
                    "return_duration": return_flight.get("total_duration"),
                    "return_segments_count": len(return_segments)
                }
                # Set the price and currency to match the return flight's (which should be the total round-trip price)
                if return_flight.get("price"):
                    combined_flight["price"] = return_flight["price"]
                if return_flight.get("currency"):
                    combined_flight["currency"] = return_flight["currency"]
                # Optionally, mark which return option this is
                combined_flight["_return_option_index"] = idx + 1
                combos.append(combined_flight)
            else:
                combos.append(outbound_flight)
        return combos
    
    def format_flight_results(self, results: Dict) -> str:
        """
        Format flight search results into a readable string with clear separation 
        of outbound and return flights for round-trip searches.
        
        Args:
            results (Dict): Raw API response from flight search
            
        Returns:
            str: Formatted flight information with enhanced round-trip display
        """
        if "error" in results:
            return f"Error: {results['error']}"
        
        output = []
        output.append("=" * 80)
        output.append("✈️  FLIGHT SEARCH RESULTS")
        output.append("=" * 80)
        
        # Search parameters
        if "search_parameters" in results:
            params = results["search_parameters"]
            output.append(f"\n📋 Search Parameters:")
            output.append(f"  Route: {params.get('departure_id', 'N/A')} → {params.get('arrival_id', 'N/A')}")
            output.append(f"  Outbound: {params.get('outbound_date', 'N/A')}")
            if params.get('return_date'):
                output.append(f"  Return: {params.get('return_date', 'N/A')}")
                output.append(f"  Trip Type: Round Trip")
            else:
                output.append(f"  Trip Type: One Way")
            output.append(f"  Currency: {params.get('currency', 'USD')}")
            if params.get('adults', 1) > 1 or params.get('children', 0) > 0 or params.get('infants', 0) > 0:
                passengers = []
                if params.get('adults', 1) > 0:
                    passengers.append(f"{params['adults']} adult(s)")
                if params.get('children', 0) > 0:
                    passengers.append(f"{params['children']} child(ren)")
                if params.get('infants', 0) > 0:
                    passengers.append(f"{params['infants']} infant(s)")
                output.append(f"  Passengers: {', '.join(passengers)}")
        
        # Price insights
        if "price_insights" in results:
            insights = results["price_insights"]
            currency = results.get('search_parameters', {}).get('currency', 'USD')
            output.append(f"\n💰 Price Insights:")
            output.append(f"  Lowest Price: {insights.get('lowest_price', 'N/A')} {currency}")
            output.append(f"  Price Level: {insights.get('price_level', 'N/A')}")
            if insights.get('typical_price_range'):
                low, high = insights['typical_price_range']
                output.append(f"  Typical Range: {low} - {high} {currency}")
        
        # Best flights
        if "best_flights" in results and results["best_flights"]:
            output.append(f"\n{'🌟'*3} BEST FLIGHTS {'🌟'*3}")
            for i, flight in enumerate(results["best_flights"], 1):
                output.append(self._format_single_flight(flight, i))
        
        # Other flights
        if "other_flights" in results and results["other_flights"]:
            output.append(f"\n{'✈️ '*3} OTHER FLIGHTS {'✈️ '*3}")
            for i, flight in enumerate(results["other_flights"], 1):
                output.append(self._format_single_flight(flight, i + len(results.get("best_flights", []))))
        
        # Summary
        total_options = len(results.get("best_flights", [])) + len(results.get("other_flights", []))
        output.append(f"\n{'='*80}")
        output.append(f"📊 Total Flight Options Found: {total_options}")
        
        # Add helpful note for round-trip flights
        if results.get('search_parameters', {}).get('return_date'):
            output.append(f"\n💡 Note: For round-trip flights, this tool automatically fetches")
            output.append(f"   both outbound (🛫) and return (🛬) flight segments.")
        
        output.append("=" * 80)
        
        return "\n".join(output)
    
    def _format_single_flight(self, flight: Dict, index: int) -> str:
        """Format a single flight option with proper separation of outbound and return flights."""
        output = []
        output.append(f"\n--- Flight Option {index} ---")
        output.append(f"Price: {flight.get('price', 'N/A')} {flight.get('currency', 'USD')}")
        output.append(f"Total Duration: {self._format_duration(flight.get('total_duration', 0))}")
        output.append(f"Type: {flight.get('type', 'N/A')}")
        
        # Carbon emissions
        if flight.get('carbon_emissions'):
            emissions = flight['carbon_emissions']
            output.append(f"Carbon Emissions: {emissions.get('this_flight', 'N/A')} kg")
            if emissions.get('difference_percent'):
                output.append(f"  ({emissions['difference_percent']:+d}% vs typical)")
        
        # Flight segments - now properly separated for round-trip flights
        if flight.get('flights'):
            if flight.get('type') == 'Round trip':
                outbound_segments, return_segments = self._separate_round_trip_segments(flight['flights'])
                
                # Display outbound flights
                if outbound_segments:
                    output.append("\n🛫 OUTBOUND FLIGHTS:")
                    for j, segment in enumerate(outbound_segments, 1):
                        output.append(self._format_flight_segment(segment, j, is_outbound=True))
                
                # Display return flights  
                if return_segments:
                    output.append("\n🛬 RETURN FLIGHTS:")
                    for j, segment in enumerate(return_segments, 1):
                        output.append(self._format_flight_segment(segment, j, is_outbound=False))
                else:
                    output.append("\n🛬 RETURN FLIGHTS:")
                    output.append("  ⚠️  Return flight details not available or not fetched")
                    if flight.get("departure_token"):
                        output.append("  💡 Try enabling auto_fetch_return_flights=True")
            else:
                # For one-way or multi-city flights, display all segments chronologically
                output.append("\n✈️  FLIGHT DETAILS:")
                for j, segment in enumerate(flight['flights'], 1):
                    output.append(self._format_flight_segment(segment, j))
        
        # Layovers
        if flight.get('layovers'):
            output.append("\n🔄 LAYOVERS:")
            for layover in flight['layovers']:
                duration_str = self._format_duration(layover.get('duration', 0))
                overnight_str = " (Overnight)" if layover.get('overnight') else ""
                output.append(f"  {layover.get('name', 'N/A')} ({layover.get('id', 'N/A')}): {duration_str}{overnight_str}")
        
        # Debug info for return flights
        if flight.get('_return_flight_info'):
            info = flight['_return_flight_info']
            output.append(f"\n🔍 Return Flight Debug Info:")
            output.append(f"  Return segments: {info.get('return_segments_count', 'N/A')}")
            output.append(f"  Return duration: {self._format_duration(info.get('return_duration', 0))}")
        
        return "\n".join(output)
    
    def _separate_round_trip_segments(self, segments: List[Dict]) -> tuple:
        """
        Separate round-trip flight segments into outbound and return flights.
        
        This method now handles the properly combined round-trip data where
        outbound and return segments are in the same flights array.
        
        Args:
            segments (List[Dict]): All flight segments (outbound + return)
            
        Returns:
            tuple: (outbound_segments, return_segments)
        """
        if not segments:
            return [], []
        
        # For properly combined round-trip flights, we need to find the turnaround point
        # This is where the journey switches from going away from origin to returning
        
        if len(segments) <= 2:
            # Simple case: likely one segment each way
            outbound_segments = [segments[0]] if segments else []
            return_segments = [segments[1]] if len(segments) > 1 else []
        else:
            # More complex case: multiple segments, need to find the break point
            
            # Strategy 1: Look for time gaps (turnaround time at destination)
            # Strategy 2: Look for airport patterns (when we start heading back)
            
            original_departure = segments[0]['departure_airport']['id']
            final_arrival = segments[-1]['arrival_airport']['id']
            
            # Find the logical split
            best_split = len(segments) // 2  # Default to middle
            
            # Look for the point where we start heading back toward origin
            for i in range(1, len(segments)):
                current_departure = segments[i]['departure_airport']['id']
                current_arrival = segments[i]['arrival_airport']['id']
                
                # If this segment heads back to the original departure airport
                # or from the final destination, this is likely the return journey start
                if (current_arrival == original_departure or 
                    current_departure == final_arrival):
                    best_split = i
                    break
                
                # Alternative: look for a time gap that indicates turnaround
                if i < len(segments):
                    prev_arrival_time = segments[i-1]['arrival_airport'].get('time', '')
                    curr_departure_time = segments[i]['departure_airport'].get('time', '')
                    
                    # If there's a significant gap (like overnight), this might be the split
                    # This is a simplified check - in practice, you'd parse the times
                    
            outbound_segments = segments[:best_split]
            return_segments = segments[best_split:]
        
        return outbound_segments, return_segments
    
    def _format_flight_segment(self, segment: Dict, segment_num: int, is_outbound: bool = None) -> str:
        """Format a single flight segment with enhanced details."""
        dep_airport = segment.get('departure_airport', {})
        arr_airport = segment.get('arrival_airport', {})
        
        output = []
        direction_emoji = "→" if is_outbound is None else ("🛫" if is_outbound else "🛬")
        
        output.append(f"  {direction_emoji} Segment {segment_num}: {segment.get('airline', 'N/A')} {segment.get('flight_number', 'N/A')}")
        output.append(f"    Route: {dep_airport.get('id', 'N/A')} ({dep_airport.get('time', 'N/A')}) → "
                     f"{arr_airport.get('id', 'N/A')} ({arr_airport.get('time', 'N/A')})")
        output.append(f"    Duration: {self._format_duration(segment.get('duration', 0))}")
        output.append(f"    Aircraft: {segment.get('airplane', 'N/A')}")
        output.append(f"    Class: {segment.get('travel_class', 'N/A')}")
        
        # Additional details
        if segment.get('legroom'):
            output.append(f"    Legroom: {segment.get('legroom')}")
        
        if segment.get('overnight'):
            output.append(f"    ⚠️  Overnight flight")
        
        if segment.get('often_delayed_by_over_30_min'):
            output.append(f"    ⚠️  Often delayed by 30+ minutes")
        
        # Extensions (amenities)
        if segment.get('extensions'):
            amenities = segment['extensions']
            if amenities:
                output.append(f"    Amenities: {', '.join(amenities[:3])}{'...' if len(amenities) > 3 else ''}")
        
        return "\n".join(output)
    
    def _format_duration(self, minutes: int) -> str:
        """Convert minutes to readable duration format."""
        if not minutes:
            return "N/A"
        hours = minutes // 60
        mins = minutes % 60
        return f"{hours}h {mins}m"

print("✅ FlightSearcher class updated with enhanced debugging and return flight fetching!")
print("💡 Key improvements:")
print("   - Added detailed debug logging to track return flight fetching")
print("   - Enhanced error handling for departure token issues")
print("   - Improved combination logic for outbound and return flights")
print("🔧 Run a new search to see the debug output and verify functionality!")

✅ FlightSearcher class updated with enhanced debugging and return flight fetching!
💡 Key improvements:
   - Added detailed debug logging to track return flight fetching
   - Enhanced error handling for departure token issues
   - Improved combination logic for outbound and return flights
🔧 Run a new search to see the debug output and verify functionality!


## 4. Initialize the Flight Searcher

Now let's create an instance of our FlightSearcher class using the API key we loaded from the environment.

In [48]:
# Re-create FlightSearcher instance with the updated class
flight_searcher = FlightSearcher()

print("✅ Updated FlightSearcher instance created successfully!")
print("🎯 New features:")
print("   - Automatic return flight fetching for round-trip searches")
print("   - Combined outbound + return flight display")
print("   - Proper segment separation with visual indicators")
print("Ready to search for complete round-trip flights! 🛫🛬")

✅ Updated FlightSearcher instance created successfully!
🎯 New features:
   - Automatic return flight fetching for round-trip searches
   - Combined outbound + return flight display
   - Proper segment separation with visual indicators
Ready to search for complete round-trip flights! 🛫🛬


## 5. Example 1: Basic Round-Trip Flight Search

Let's start with a simple example: searching for round-trip flights from Los Angeles (LAX) to New York JFK for a future date.

In [50]:
# Example 1: Basic round-trip flight search
print("🔍 Searching for flights from AUS to JFK...")

try:
    # Search for round-trip flights
    results = flight_searcher.search_flights(
        departure_id="AUS",
        arrival_id="JFK", 
        outbound_date="2025-09-15",  # Future date
        return_date="2025-09-22",    # One week later
        trip_type="round_trip",
        adults=1,
        travel_class="economy",
        stops=0
    )
    
    # Format and display results
    formatted_results = flight_searcher.format_flight_results(results)
    print(formatted_results)
    
    print("\n✅ Basic flight search completed successfully!")
    
except Exception as e:
    print(f"❌ Error during flight search: {e}")
    print("This might be due to API limits or network issues.")

🔍 Searching for flights from AUS to JFK...
✈️  FLIGHT SEARCH RESULTS

📋 Search Parameters:
  Route: AUS → JFK
  Outbound: 2025-09-15
  Return: 2025-09-22
  Trip Type: Round Trip
  Currency: USD

💰 Price Insights:
  Lowest Price: 158 USD
  Price Level: low
  Typical Range: 175 - 285 USD

🌟🌟🌟 BEST FLIGHTS 🌟🌟🌟

--- Flight Option 1 ---
Price: 173 USD
Total Duration: 15h 13m
Type: Round trip
Carbon Emissions: 192000 kg
  (-16% vs typical)

🛫 OUTBOUND FLIGHTS:
  🛫 Segment 1: Frontier F9 4702
    Route: AUS (2025-09-15 17:29) → ATL (2025-09-15 20:54)
    Duration: 2h 25m
    Aircraft: Airbus A321neo
    Class: Economy
    Legroom: 28 in
    ⚠️  Often delayed by 30+ minutes
    Amenities: Below average legroom (28 in), Carbon emissions estimate: 96 kg
  🛫 Segment 2: Frontier F9 4818
    Route: ATL (2025-09-15 22:46) → JFK (2025-09-16 01:08)
    Duration: 2h 22m
    Aircraft: Airbus A321neo
    Class: Economy
    Legroom: 28 in
    ⚠️  Often delayed by 30+ minutes
    Amenities: Below average l

In [46]:
import os, requests
base = "https://serpapi.com/search"
params = {
  "engine": "google_flights",
  "api_key": os.getenv("SERPAPI_API_KEY"),
  "departure_id": "AUS",
  "arrival_id": "JFK",
  "outbound_date": "2025-09-15",
  "return_date": "2025-09-22",
  "type": "1",                 # round-trip
  "travel_class": "1",   # string, not numeric
  "adults": "1",
  "currency": "USD",
  "hl": "en",
  "gl": "us",
  "deep_search": "true"        # lowercase string
}
r = requests.get(base, params=params)
print(r.status_code, r.url)
print(r.text)

200 https://serpapi.com/search?engine=google_flights&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&departure_id=AUS&arrival_id=JFK&outbound_date=2025-09-15&return_date=2025-09-22&type=1&travel_class=1&adults=1&currency=USD&hl=en&gl=us&deep_search=true
{
  "search_metadata": {
    "id": "68b6ff8102ab8ca0555a5c99",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/2107ecc52753e4c9/68b6ff8102ab8ca0555a5c99.json",
    "created_at": "2025-09-02 14:30:25 UTC",
    "processed_at": "2025-09-02 14:30:25 UTC",
    "google_flights_url": "https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA5LTE1agcIARIDQVVTcgcIARIDSkZLGh4SCjIwMjUtMDktMjJqBwgBEgNKRktyBwgBEgNBVVNCAQFIAXABmAEB&tfu=EgIIAQ",
    "raw_html_file": "https://serpapi.com/searches/2107ecc52753e4c9/68b6ff8102ab8ca0555a5c99.html",
    "prettify_html_file": "https://serpapi.com/searches/2107ecc52753e4c9/68b6ff8102ab8ca0555a5c99.prettify",
    "total_time_taken": 

In [24]:
# Let's examine the structure of the results to understand the departure_token issue
print("🔍 Inspecting the flight data structure...")
print("=" * 60)

# Check if we have results and what the structure looks like
if 'results' in locals() and results:
    print("📊 Search Parameters:")
    if "search_parameters" in results:
        for key, value in results["search_parameters"].items():
            print(f"  {key}: {value}")
    
    print(f"\n📋 Available top-level keys in results:")
    for key in results.keys():
        print(f"  - {key}")
    
    # Check the first flight option structure
    if "best_flights" in results and results["best_flights"]:
        first_flight = results["best_flights"][0]
        print(f"\n✈️  First flight structure:")
        for key, value in first_flight.items():
            if key == "flights":
                print(f"  {key}: {len(value)} segments")
                for i, segment in enumerate(value):
                    dep_airport = segment.get('departure_airport', {})
                    arr_airport = segment.get('arrival_airport', {})
                    print(f"    Segment {i+1}: {dep_airport.get('id', 'N/A')} → {arr_airport.get('id', 'N/A')} ({dep_airport.get('time', 'N/A')} - {arr_airport.get('time', 'N/A')})")
            elif key == "departure_token":
                print(f"  🎟️  {key}: {value[:50]}..." if isinstance(value, str) and len(value) > 50 else f"  🎟️  {key}: {value}")
            else:
                print(f"  {key}: {value}")
    
    print(f"\n💡 Key Finding:")
    if "best_flights" in results and results["best_flights"] and "departure_token" in results["best_flights"][0]:
        print("✅ departure_token found! This means we need a second API call to get return flights.")
        print("❌ Current implementation only shows outbound flights - this explains the missing return flights!")
    else:
        print("❌ No departure_token found in the data structure.")
        
else:
    print("❌ No results available. Please run the search first.")

🔍 Inspecting the flight data structure...
📊 Search Parameters:

📋 Available top-level keys in results:
  - error

💡 Key Finding:
❌ No departure_token found in the data structure.


In [25]:
# Test the updated round-trip functionality
print("🧪 Testing updated round-trip flight search with automatic return flight fetching...")
print("=" * 80)

try:
    # Search with the updated functionality
    updated_results = flight_searcher.search_flights(
        departure_id="DFW",          # Dallas
        arrival_id="LAX",            # Los Angeles
        outbound_date="2025-07-20",  # Future date
        return_date="2025-07-27",    # One week later
        trip_type="round_trip",
        adults=1,
        travel_class="economy",
        max_price=600,               # Reasonable price limit
        auto_fetch_return_flights=True  # Enable automatic return flight fetching
    )
    
    # Format and display the results
    formatted_updated = flight_searcher.format_flight_results(updated_results)
    print(formatted_updated)
    
    print("\n✅ Updated round-trip search completed!")
    print("🎯 This should now show BOTH outbound AND return flights!")
    
except Exception as e:
    print(f"❌ Error during updated flight search: {e}")
    print("This might be due to API limits or network issues.")
    import traceback
    traceback.print_exc()

🧪 Testing updated round-trip flight search with automatic return flight fetching...
❌ Error during updated flight search: API request failed: 400 Client Error: Bad Request for url: https://serpapi.com/search?engine=google_flights&departure_id=DFW&arrival_id=LAX&outbound_date=2025-07-20&currency=USD&hl=en&gl=us&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&type=1&return_date=2025-07-27&travel_class=1&adults=1&max_price=600
This might be due to API limits or network issues.


Traceback (most recent call last):
  File "C:\Users\armoshar\AppData\Local\Temp\ipykernel_7776\628613071.py", line 199, in _search_flights_single
    response.raise_for_status()
  File "c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\requests\models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://serpapi.com/search?engine=google_flights&departure_id=DFW&arrival_id=LAX&outbound_date=2025-07-20&currency=USD&hl=en&gl=us&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&type=1&return_date=2025-07-27&travel_class=1&adults=1&max_price=600

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\armoshar\AppData\Local\Temp\ipykernel_7776\2912171420.py", line 7, in <module>
    updated_results = flight_searcher.search_flights(
                      ^^^^^^^^^^^^^^^^^^^^

## 6. Example 2: Advanced Search with Filters

Now let's try a more advanced search with multiple filters including specific airlines, price limits, and time preferences.

In [7]:
# Example 2: Advanced search with multiple filters
print("🔍 Searching for business class flights with filters...")

try:
    # Advanced search with multiple parameters
    results = flight_searcher.search_flights(
        departure_id="AUS",          # San Francisco
        arrival_id="FCO",            # London Heathrow
        outbound_date="2025-08-01",  # Future date
        return_date="2025-08-10",    
        trip_type="round_trip",
        adults=3,
        children=1,                   # Two passengers
        travel_class="economy",     # Business class
        departure_time_range="6,18", # Depart between 6AM-6PM
        return_time_range="10,22",   # Return between 10AM-10PM
        #max_price=5000,              # Maximum $5000
        include_airlines=["BA", "VS", "UA"],  # British Airways, Virgin, United
        min_layover_duration=60,     # Minimum 1 hour layover
        max_layover_duration=300,    # Maximum 5 hours layover
        deep_search=True             # More accurate results
    )
    
    # Format and display results
    formatted_results = flight_searcher.format_flight_results(results)
    print(formatted_results)
    
    print("\n✅ Advanced flight search completed successfully!")
    
except Exception as e:
    print(f"❌ Error during advanced flight search: {e}")
    print("This might be due to API limits, network issues, or strict filter criteria.")

🔍 Searching for business class flights with filters...
❌ Error during advanced flight search: API request failed: 400 Client Error: Bad Request for url: https://serpapi.com/search.json?engine=google_flights&departure_id=AUS&arrival_id=FCO&outbound_date=2025-08-01&currency=USD&hl=en&gl=us&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&type=1&return_date=2025-08-10&travel_class=1&adults=3&children=1&outbound_times=6%2C18&return_times=10%2C22&layover_duration=60%2C300&include_airlines=BA%2CVS%2CUA&deep_search=true
This might be due to API limits, network issues, or strict filter criteria.


## 7. Example 3: Round-Trip with Enhanced Formatting

Let's do a round-trip search to specifically demonstrate the improved formatting that clearly separates outbound and return flights.

In [8]:
# Example 3: Round-trip with enhanced outbound/return formatting
print("🔍 Searching for round-trip flights with enhanced formatting...")
print("This example demonstrates the improved display of outbound vs return flights")

try:
    # Round-trip search to demonstrate enhanced formatting
    results = flight_searcher.search_flights(
        departure_id="DFW",          # Dallas/Fort Worth
        arrival_id="LAX",            # Los Angeles
        outbound_date="2025-09-15",  # Future date
        return_date="2025-09-22",    # One week later
        trip_type="round_trip",
        adults=1,
        travel_class="economy",
        max_price=800                # Reasonable price limit
    )
    
    # Format and display results with enhanced round-trip formatting
    formatted_results = flight_searcher.format_flight_results(results)
    print(formatted_results)
    
    print("\n✅ Round-trip search with enhanced formatting completed!")
    print("📝 Notice how outbound (🛫) and return (🛬) flights are clearly separated")
    
except Exception as e:
    print(f"❌ Error during round-trip flight search: {e}")
    print("This might be due to API limits or network issues.")

🔍 Searching for round-trip flights with enhanced formatting...
This example demonstrates the improved display of outbound vs return flights
✈️  FLIGHT SEARCH RESULTS

📋 Search Parameters:
  Route: DFW → LAX
  Outbound: 2025-09-15
  Return: 2025-09-22
  Trip Type: Round Trip
  Currency: USD

💰 Price Insights:
  Lowest Price: 144 USD
  Price Level: typical
  Typical Range: 110 - 155 USD

🌟🌟🌟 BEST FLIGHTS 🌟🌟🌟

--- Flight Option 1 ---
Price: 148 USD
Total Duration: 6h 23m
Type: Round trip
Carbon Emissions: 178000 kg
  (+5% vs typical)

🛫 OUTBOUND FLIGHTS:
  🛫 Segment 1: Spirit NK 1513
    Route: DFW (2025-09-15 14:54) → LAX (2025-09-15 16:12)
    Duration: 3h 18m
    Aircraft: Airbus A320
    Class: Economy
    Legroom: 28 in
    Amenities: Below average legroom (28 in), Wi-Fi for a fee, Carbon emissions estimate: 177 kg

🛬 RETURN FLIGHTS:
  🛬 Segment 1: Spirit NK 2310
    Route: LAX (2025-09-22 00:45) → DFW (2025-09-22 05:50)
    Duration: 3h 5m
    Aircraft: Airbus A320
    Class: Econom

## 8. Example 4: One-Way Flight with Family

Let's search for a one-way flight suitable for a family with children and specific requirements.

In [9]:
# Example 4: One-way family flight search
print("🔍 Searching for one-way family flights...")

try:
    # Family flight search
    results = flight_searcher.search_flights(
        departure_id="MIA",          # Miami
        arrival_id="ORD",            # Chicago O'Hare
        outbound_date="2025-07-04",  # Independence Day
        trip_type="one_way",         # One-way ticket
        adults=2,                    # 2 adults
        children=2,                  # 2 children
        infants=1,                   # 1 infant
        travel_class="economy",      # Economy for budget
        departure_time_range="8,16", # Morning to afternoon departure
        max_price=800,               # Budget-friendly
        max_duration=300,            # Max 5 hours total travel time
        exclude_airlines=["NK", "F9"] # Avoid ultra-low-cost carriers
    )
    
    # Format and display results
    formatted_results = flight_searcher.format_flight_results(results)
    print(formatted_results)
    
    print("\n✅ Family flight search completed successfully!")
    
except Exception as e:
    print(f"❌ Error during family flight search: {e}")
    print("This might be due to API limits or no available flights matching criteria.")

🔍 Searching for one-way family flights...
❌ Error during family flight search: API request failed: 400 Client Error: Bad Request for url: https://serpapi.com/search.json?engine=google_flights&departure_id=MIA&arrival_id=ORD&outbound_date=2025-07-04&currency=USD&hl=en&gl=us&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&type=2&travel_class=1&adults=2&children=2&infants_in_seat=1&outbound_times=8%2C16&max_price=800&max_duration=300&exclude_airlines=NK%2CF9
This might be due to API limits or no available flights matching criteria.


## 8. Utility Functions and Tips

Here are some utility functions and tips for working with the flight search results.

In [10]:
def save_search_results(results, filename=None):
    """Save search results to a JSON file."""
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"flight_search_results_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"💾 Results saved to: {filename}")
    return filename

def extract_best_deals(results, max_results=3):
    """Extract the best flight deals from search results."""
    best_deals = []
    
    if "best_flights" in results:
        for flight in results["best_flights"][:max_results]:
            deal = {
                "price": flight.get("price", "N/A"),
                "currency": flight.get("currency", "USD"),
                "duration": flight_searcher._format_duration(flight.get("total_duration", 0)),
                "stops": len(flight.get("flights", [])) - 1,
                "airline": flight.get("flights", [{}])[0].get("airline", "N/A") if flight.get("flights") else "N/A"
            }
            best_deals.append(deal)
    
    return best_deals

def compare_prices(results_list, route_names):
    """Compare prices across multiple search results."""
    print("💰 Price Comparison:")
    print("-" * 50)
    
    for i, (results, route_name) in enumerate(zip(results_list, route_names)):
        if "best_flights" in results and results["best_flights"]:
            best_price = results["best_flights"][0].get("price", "N/A")
            currency = results["best_flights"][0].get("currency", "USD")
            print(f"{route_name}: {best_price} {currency}")
        else:
            print(f"{route_name}: No flights found")

print("✅ Utility functions defined!")
print("\nTips for better flight searches:")
print("🎯 Use flexible dates (+/- 3 days) for better prices")
print("✈️  Consider nearby airports for more options")
print("⏰ Search 6-8 weeks in advance for domestic flights")
print("🌍 Search 2-3 months in advance for international flights")
print("💡 Use deep_search=True for more accurate results (slower)")
print("🔄 Try different combinations of filters for best results")

✅ Utility functions defined!

Tips for better flight searches:
🎯 Use flexible dates (+/- 3 days) for better prices
✈️  Consider nearby airports for more options
⏰ Search 6-8 weeks in advance for domestic flights
🌍 Search 2-3 months in advance for international flights
💡 Use deep_search=True for more accurate results (slower)
🔄 Try different combinations of filters for best results


## 9. Enhanced Round-Trip Flight Display

### 🎯 Key Improvement: Clear Outbound vs Return Flight Separation

One of the major improvements in this notebook is the enhanced formatting for round-trip flights. Here's what's new:

#### ✨ **Before**: All segments listed chronologically
```
Flight Details:
  Segment 1: United 1234 (DEP → HUB)
  Segment 2: United 5678 (HUB → DEST)  
  Segment 3: Delta 9012 (DEST → HUB2)
  Segment 4: Delta 3456 (HUB2 → DEP)
```

#### 🌟 **After**: Clear separation of outbound and return journeys
```
🛫 OUTBOUND FLIGHTS:
  🛫 Segment 1: United 1234 (DEP → HUB)
  🛫 Segment 2: United 5678 (HUB → DEST)

🛬 RETURN FLIGHTS:
  🛬 Segment 1: Delta 9012 (DEST → HUB2)  
  🛬 Segment 2: Delta 3456 (HUB2 → DEP)
```

### 🧠 **How It Works**

The enhanced formatting uses intelligent algorithms to:

1. **Detect round-trip flights** based on the `type` field from the API
2. **Analyze flight segments** to find the logical break point between outbound and return
3. **Separate segments** into outbound (🛫) and return (🛬) categories
4. **Display clearly** with visual indicators and better organization

### 💡 **Smart Segment Separation**

The algorithm considers:
- **Airport analysis**: Detecting when flights start heading back to origin
- **Geographic logic**: Understanding flight routing patterns  
- **Chronological flow**: Finding natural breaks in the journey
- **API structure**: Leveraging Google Flights' segment organization

### 📊 **Benefits**

✅ **Easier to understand** complex multi-segment round-trip flights  
✅ **Better planning** - see outbound and return journeys separately  
✅ **Visual clarity** with emojis and clear section headers  
✅ **Enhanced details** including amenities, legroom, and warnings  
✅ **Smart layover display** with overnight indicators  

### 🔧 **Technical Implementation**

Key methods added:
- `_separate_round_trip_segments()`: Intelligently splits segments
- `_format_flight_segment()`: Enhanced segment formatting with details
- Updated `_format_single_flight()`: Route detection and better organization
- Enhanced `format_flight_results()`: Improved headers and summary info

This makes the flight search results much more readable and user-friendly, especially for complex international round-trip flights with multiple connections!

## 10. Conclusion

This notebook demonstrated how to use the Google Flights Search Tool with SerpApi to:

✅ **Load environment variables securely** using `load_dotenv()`  
✅ **Handle directory structure** properly (notebook in subdirectory)  
✅ **Search for flights** with various parameters and filters  
✅ **Format and display results** with enhanced round-trip visualization  
✅ **Clearly separate outbound and return flights** for better readability  
✅ **Handle different scenarios** (round-trip, one-way, family travel)  
✅ **Use utility functions** for saving and comparing results  

### 🔧 Directory Structure Solution

We solved the import path issue by:
- Including the complete FlightSearcher implementation directly in this notebook
- Adding proper path handling for notebooks in subdirectories
- Creating a self-contained, fully functional flight search environment

### Next Steps

- ✈️ **Experiment** with different airports and date combinations
- 🎯 **Try various filter combinations** to find the best deals
- 💾 **Use the utility functions** to save and analyze your search results
- 🔄 **Run searches periodically** for price monitoring
- 📊 **Customize the output formatting** for your specific needs

### 🛠️ Development Notes

If you want to create a standalone Python script:
1. Copy the FlightSearcher class from this notebook
2. Save it as a separate `.py` file
3. Import it in your scripts with proper path handling

### Remember

- ✅ **API key is loaded from .env** - secure and convenient
- ⚡ **Everything works within this notebook** - no import issues
- 🔒 **Be mindful of API rate limits** when running multiple searches
- 📈 **Flight prices change frequently** - results may vary
- 🌐 **Use `deep_search=True`** for more accurate results when needed

### 🚀 Ready to Fly!

You now have a complete, working flight search tool that:
- Loads API keys securely with dotenv
- Works perfectly in a Jupyter notebook environment  
- Handles all major flight search scenarios
- **Provides enhanced round-trip flight formatting** with clear outbound/return separation
- Shows detailed flight information with amenities and warnings
- Provides beautiful, formatted output with visual indicators

Happy flight searching! ✈️🌍

---

**🏆 Pro Tip**: Bookmark this notebook and use it as your go-to flight search tool. Modify the examples to create your own custom searches!

In [ ]:
from serpapi import GoogleSearch
import json

def fetch_round_trip_with_multi_city(api_key, outbound_params):
    multi_city = [
        {
            "departure_id": outbound_params["departure_id"],
            "arrival_id": outbound_params["arrival_id"],
            "date": outbound_params["outbound_date"]
        },
        {
            "departure_id": outbound_params["arrival_id"],  # reversed
            "arrival_id": outbound_params["departure_id"],
            "date": outbound_params["return_date"]
        }
    ]
    params = {
        "engine": "google_flights",
        "type": "3",
        "multi_city_json": json.dumps(multi_city),
        "adults": outbound_params.get("adults", 1),
        "travel_class": outbound_params.get("travel_class", "1"),
        "currency": outbound_params.get("currency", "USD"),
        "hl": outbound_params.get("hl", "en"),
        "api_key": os.getenv("SERPAPI_API_KEY", api_key)
    }
    search = GoogleSearch(params)
    return search.get_dict()

SERPAPI_API_KEY = ""
outbound_params = {
    "departure_id": "AUS",
    "arrival_id": "JFK",
    "outbound_date": "2025-07-15",
    "return_date": "2025-07-22",
    "adults": 1,
    "travel_class": "1",
    "currency": "USD",
    "hl": "en"
}

results = fetch_round_trip_with_multi_city(SERPAPI_API_KEY, outbound_params)

if "best_flights" in results and results["best_flights"]:
    for i, flight in enumerate(results["best_flights"], 1):
        flights_list = flight.get("flights", [])
        print(f"\nROUND-TRIP OPTION {i}:  Price: ${flight['price']}")
        if len(flights_list) == 0:
            print("  [!] No flight segments found for this option.")
            continue
        # Try to infer where the outbound leg ends and the return begins
        # We'll separate by departure airport: outbound from AUS, return from JFK
        outbound = []
        inbound = []
        in_return = False
        for segment in flights_list:
            if segment["departure_airport"]["id"] == outbound_params["departure_id"]:
                in_return = False  # Outbound
            elif segment["departure_airport"]["id"] == outbound_params["arrival_id"]:
                in_return = True   # Return
            if in_return:
                inbound.append(segment)
            else:
                outbound.append(segment)
        # Print outbound
        print("  OUTBOUND:")
        for seg in outbound:
            dep = seg["departure_airport"]
            arr = seg["arrival_airport"]
            print(f"    {dep['name']} ({dep['id']}) {dep['time']} → {arr['name']} ({arr['id']}) {arr['time']}")
        # Print inbound (return)
        if inbound:
            print("  RETURN:")
            for seg in inbound:
                dep = seg["departure_airport"]
                arr = seg["arrival_airport"]
                print(f"    {dep['name']} ({dep['id']}) {dep['time']} → {arr['name']} ({arr['id']}) {arr['time']}")
        else:
            print("  [!] No return segment found for this option.")
else:
    print("No round-trip flights found or API returned an error.")
    print(json.dumps(results, indent=2))


No round-trip flights found or API returned an error.
{
  "error": "Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key"
}


In [12]:
# After getting return_results
import json

if "best_flights" in return_results and return_results["best_flights"]:
    for i, flight in enumerate(return_results["best_flights"], 1):
        print(f"\nRETURN FLIGHT OPTION {i}:")
        for segment in flight["flights"]:
            dep = segment["departure_airport"]
            arr = segment["arrival_airport"]
            print(f"  {dep['name']} ({dep['id']}) {dep['time']} → {arr['name']} ({arr['id']}) {arr['time']}")
        print(f"  Price: ${flight['price']}")
else:
    print("No return flights found or API returned an error.")
    print(json.dumps(return_results, indent=2))  # <-- Add this line


NameError: name 'return_results' is not defined

In [ ]:
from serpapi import GoogleSearch
import json

multi_city = [
    {
        "departure_id": "AUS",
        "arrival_id": "JFK",
        "date": "2025-07-15"
    },
    {
        "departure_id": "JFK",
        "arrival_id": "AUS",
        "date": "2025-07-22"
    }
]

params = {
    "engine": "google_flights",
    "type": "3",  # Multi-city
    "multi_city_json": json.dumps(multi_city),
    "adults": 1,
    "travel_class": "1",
    "currency": "USD",
    "hl": "en",
    "api_key": os.getenv("SERPAPI_API_KEY")  # <-- Put your API key here
}

search = GoogleSearch(params)
results = search.get_dict()

import json
if "best_flights" in results and results["best_flights"]:
    for i, flight in enumerate(results["best_flights"], 1):
        flights_list = flight.get("flights", [])
        print(f"\nROUND-TRIP OPTION {i}:  Price: ${flight['price']}")
        if not flights_list:
            print("  [!] No flight segments found for this option.")
            continue
        # Separate outbound/return
        outbound = []
        inbound = []
        in_return = False
        for segment in flights_list:
            if segment["departure_airport"]["id"] == "AUS":
                in_return = False
            elif segment["departure_airport"]["id"] == "JFK":
                in_return = True
            if in_return:
                inbound.append(segment)
            else:
                outbound.append(segment)
        print("  OUTBOUND:")
        for seg in outbound:
            dep = seg["departure_airport"]
            arr = seg["arrival_airport"]
            print(f"    {dep['name']} ({dep['id']}) {dep['time']} → {arr['name']} ({arr['id']}) {arr['time']}")
        if inbound:
            print("  RETURN:")
            for seg in inbound:
                dep = seg["departure_airport"]
                arr = seg["arrival_airport"]
                print(f"    {dep['name']} ({dep['id']}) {dep['time']} → {arr['name']} ({arr['id']}) {arr['time']}")
        else:
            print("  [!] No return segment found for this option.")
else:
    print("No round-trip flights found or API returned an error.")
    print(json.dumps(results, indent=2))


In [ ]:
import requests

api_key = os.getenv('SERPAPI_API_KEY')
test_url = f"https://serpapi.com/account?api_key={api_key}"

response = requests.get(test_url)
if response.status_code == 200:
    print("✅ API key is valid!")
    data = response.json()
    print(f"Account email: {data.get('account_email')}")
    print(f"API searches left: {data.get('searches_left')}")
else:
    print(f"❌ API key validation failed: {response.status_code}")

❌ API key validation failed: 401
